# 理论和实践，要相结合。Coding! Coding! Coding!
## 林彪元帅说学习毛主席语录要“带着问题学，活学活用，学用结合，急用先学，立竿见影”。


---

### 因子实战 第十集 
## 因子回测 Backtesting 系列 代码习题课


🌟🌟🌟 欢迎来到哈罗德的量化频道！🌟🌟🌟

### 🎬 主讲：大导演哈罗德
- 学历背景：香港中文大学(深圳)本科学位 金融工程专业
- 下一步学业：即将前往美国纽约进修金融工程硕士（已获得录取）
- 🌐 [关注我的Bilibili，看所有人都能听得懂的量化学习内容](https://space.bilibili.com/629573485)
- 📚 [关注我的YouTube，看更多量化投资干货！](https://www.youtube.com/@BD_Harold)
- 📱 微信公众号：大导演哈罗德，所有的数据都可以在这个获取，只需回复 因子实战 + # （例如 因子实战6 ）
🌟🌟🌟 量化不是束之高阁的灵丹妙药，量化是让散户投资者认识市场的工具 #哈罗德的量化频道 🌟🌟🌟


---

In [25]:
"""
这个框框的代码是用来导入数据的和因子回测的方程的。
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from 因子回测包 import factor_correlation as FactorIC
from 因子回测包 import factor_group as FactorGroup

sw_ind = pd.read_pickle('数据/IndexComponent_SWN_I.txt')
stock_close = pd.read_pickle('数据/StockQuote_ClosePrice_BackwardAdj.txt')
stock_open = pd.read_pickle('数据/StockQuote_OpenPrice_BackwardAdj.txt')
monthly_trading_day = pd.read_pickle('数据/monthly_trading_day.pkl')
monthly_trading_day['start_date'] = pd.to_datetime(monthly_trading_day['start_date'], format='%Y%m%d')
monthly_trading_day['end_date'] = pd.to_datetime(monthly_trading_day['end_date'], format='%Y%m%d')
start_date = pd.to_datetime('20120101', format='%Y%m%d')
end_date = pd.to_datetime('20231231', format='%Y%m%d')
filtered_trading_days = monthly_trading_day.loc[(monthly_trading_day['end_date'] >= start_date) & (monthly_trading_day['end_date'] <= end_date)]
stock_close.index = pd.to_datetime(stock_close.index)
stock_open.index = pd.to_datetime(stock_open.index)
stock_ret_monthly = stock_close.reindex(index = filtered_trading_days.end_date).pct_change() 
stock_ret_monthly_nextopen = stock_open.shift(-1).reindex(filtered_trading_days.end_date).pct_change()

def simple_factor_test(factor,use_data = 'this_close'):
    if use_data=='this_close':
        # 
        this_ret_data = stock_ret_monthly.shift(-1)
    else: 
        this_ret_data = stock_ret_monthly_nextopen.shift(-1)

    ic,rankic = FactorIC(factor,this_ret_data)
    factor_group = FactorGroup(factor)
    condata = pd.concat([factor_group.unstack(),this_ret_data.unstack()],axis=1).dropna().reset_index()
    condata.columns =['stockcode','date','group_id','ret']
    group_ret = condata.groupby(['date','group_id'])['ret'].mean().unstack()
    return ic,rankic,group_ret

pe = pd.read_pickle('数据/StockQuote_PEttm.txt')
ep = 1/pe
ep.replace(np.inf, np.nan, inplace = True) #  计算EPttm并去除无限大的值
ep.replace(-np.inf, np.nan, inplace = True) #  计算EPttm并去除无限大的值
dEP = ep.diff(60) # dEP因子是EP因子的六十日delta
# 回测周期：2012-01-01至2023-12-31
# change index into datetime
dEP.index = pd.to_datetime(dEP.index)
dEP_monthly = dEP.reindex(index = filtered_trading_days.end_date) # 按月抽样得到dEP的因子值

ic,rankic,group_ret  = simple_factor_test(dEP_monthly)

In [26]:
dEP_monthly

,000001.SZ,000002.SZ,000003.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,...,873570.BJ,873576.BJ,873593.BJ,873665.BJ,873679.BJ,873693.BJ,873703.BJ,873726.BJ,873833.BJ,T00018.SH
end_date,,,,,,,,,,,,,,,,,,,,,
2012-01-31,0.002481,0.002968,NaN,-0.005549,-0.000836,0.003023,0.001183,0.000250,0.003496,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-02-29,-0.008819,-0.014278,NaN,0.000517,0.000000,-0.048998,-0.006231,0.000168,0.006528,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-30,0.009866,0.011612,NaN,-0.000039,0.000000,-0.075154,-0.010442,NaN,0.002593,0.000081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-04-27,0.024902,0.009117,NaN,0.000090,-0.011260,-0.026038,-0.008362,NaN,-0.005099,0.003222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-31,0.035999,0.013286,NaN,0.000321,-0.011072,-0.006224,-0.000973,NaN,-0.006213,0.003018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-31,0.000320,-0.006076,NaN,NaN,NaN,-0.005207,NaN,NaN,-0.001222,NaN,...,NaN,NaN,-0.002515,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-31,0.011596,-0.010884,NaN,NaN,NaN,-0.030556,NaN,NaN,0.005729,NaN,...,NaN,NaN,0.009349,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-28,0.016771,0.002706,NaN,NaN,NaN,-0.021728,NaN,NaN,0.004374,NaN,...,NaN,0.011647,0.005304,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# TODO
### 跟着我的视频！让我们一点一点完成吧！！！！

In [ ]:
print("hello world")

In [ ]:
def FactorGroup(factor,split_method='average',split_num=5,industry_factor=None,limit_df=None):
	'''
	将因子进行分类,按照行分类。
	:param factor:要分类的因子,或者打分
	:param split_method:默认为等比例分组,'average',还可以有'largest','smallest','largest_ratio','smallest_ratio'
	:param split_num:若split_method=='average',则等分split_num组,若为'largest',则最大n个,若'smallest',则最小n个,若largest_ratio,则最大百分比,若smallest_ratio,则最小百分比
	:param industry_factor(pd.DataFrame or None):行业因子
	:param limit_df(pd.DataFrame or NOne):None或者TrueFalse构成的df,来自于FactorTool_GetLimitDf的结果
	:return:factor_split_result, df
	'''
	if limit_df is not None:
		[factor,limit_df] = align_factor([factor,limit_df])
		limit_df = limit_df.fillna(value=True).astype('bool')
		factor = factor[limit_df]
		
	if industry_factor is None:
		industry_factor = pd.DataFrame(index=factor.index,columns=factor.columns,data='Market')
		industry_factor = industry_factor[factor.notnull()].astype('object')
	else:
		[factor,industry_factor] = align_factor([factor,industry_factor])
		industry_factor = industry_factor.astype('object')
		industry_factor = industry_factor.fillna(value='others')
		industry_factor = industry_factor[factor.notnull()]
		
	data = pd.DataFrame(index=pd.MultiIndex.from_product([factor.index,factor.columns],names=['date','asset']))
	data['group'] = industry_factor.stack()
	data['factor'] = factor.stack()
	data = data.dropna(subset=['group'])
	
	data_factor_array = data['factor'].values
	data_final_split = np.full((len(data_factor_array),),np.nan)
	
	grouper = [data.index.get_level_values('date'),'group']
	data_groupby = data.groupby(grouper)
	data_groupby_indices = data_groupby.indices
	data_groupby_indices = list(data_groupby_indices.values())
	
	def auxilary_get_split_array(data_factor_array,data_final_split,data_groupby_indices,split_method,split_num):
		def quantile_split(_this_split_result,_this_array,_split_percentile):
			split_value = np.nanpercentile(_this_array,_split_percentile)
			split_value[0] -= 1
			split_value[-1] += 1
			for i in range(len(split_value)-1):
				_this_split_result[(_this_array<=split_value[i+1])&(_this_array>split_value[i])] = i
			return _this_split_result
			
		if split_method=='average':
			split_percentile = np.linspace(0,100,split_num+1)
		elif split_method=='largest_ratio':
			split_percentile = np.array([0,100-split_num*100,100])
		elif split_method=='smallest_ratio':
			split_percentile = np.array([0,split_num*100,100])
		
		for this_group_place in range(len(data_groupby_indices)):
			this_indice_place = data_groupby_indices[this_group_place]
			this_factor_array = data_factor_array[this_indice_place]
			this_split_result = data_final_split[this_indice_place]
			# if split_method in ['average','largest','smallest']:
			if split_method =='average':
				this_data_final_split = quantile_split(this_split_result,this_factor_array,split_percentile)
				data_final_split[this_indice_place] = this_data_final_split
			elif split_method=='smallest':
				this_factor_array_sort = np.sort(this_factor_array[~np.isnan(this_factor_array)])
				split_value = this_factor_array_sort[min(len(this_factor_array_sort)-1,split_num-1)]			
				if len(split_value)>0:
					this_split_result[this_factor_array<=split_value]=0
					this_split_result[this_factor_array>split_value]=1
					data_final_split[this_indice_place] = this_split_result
			elif split_method=='largest':
				this_factor_array_sort = np.sort(this_factor_array[~np.isnan(this_factor_array)])[::-1]
				split_value = this_factor_array_sort[min(len(this_factor_array_sort)-1,split_num-1)]
				if len(split_value)>0:
					this_split_result[this_factor_array<split_value] = 0
					this_split_result[this_factor_array>=split_value] = 1
					data_final_split[this_indice_place] = this_split_result
		return data_final_split
	
	data_final_split = auxilary_get_split_array(data_factor_array,data_final_split,data_groupby_indices,split_method,split_num)	
	data.loc[:,'factor'] = data_final_split	
	final_data = data['factor'].unstack().reindex(index=factor.index,columns=factor.columns)
	
	return final_data